In [1]:
import rpy2.robjects as robjects
from rpy2.robjects import numpy2ri
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
import numpy as np
import pandas as pd

In [2]:
robjects.r['source']('../ratio_estimator/ratio/auxs_funcs.R')  

R callback write-console: randomForest 4.7-1.2
  
R callback write-console: Type rfNews() to see new features/changes/bug fixes.
  
R callback write-console: Loading required package: spam
  
R callback write-console: Spam version 2.11-1 (2025-01-20) is loaded.
Type 'help( Spam)' or 'demo( spam)' for a short introduction 
and overview of this package.
Help for individual functions is also obtained by adding the
suffix '.spam' to the function name, e.g. 'help( chol.spam)'.
  
R callback write-console: 
Attaching package: ‘spam’

  
R callback write-console: The following objects are masked from ‘package:base’:

    backsolve, forwardsolve

  
R callback write-console: Loading required package: viridisLite
  
R callback write-console: 
Try help(fields) to get started.
  
R callback write-console: 
Attaching package: ‘FNN’

  
R callback write-console: The following objects are masked from ‘package:class’:

    knn, knn.cv

  


value,[3]
visible,[10]


In [3]:
g_forman_random = robjects.globalenv['g.forman.random']

In [4]:
def generate_sample_fig1(scale):
    n_source_plus = int(200*scale)
    n_source_minus = int(150*scale)
    n_source = n_source_plus + n_source_minus
    pi_source = n_source_plus/n_source

    n_target_plus = int(25*scale)
    n_target_minus = int(75*scale)
    n_target = n_target_plus + n_target_minus
    pi_target = n_target_plus/n_target

    p_source_plus = np.random.multivariate_normal(mean=np.array([0,0]), cov=np.eye(2), size=(n_source_plus))
    p_source_minus = np.random.multivariate_normal(mean=np.array([2,1]), cov=np.array([[2,1], [1,2]]), size=(n_source_minus))

    p_target_plus = np.random.multivariate_normal(mean=np.array([0,0]), cov=np.eye(2), size=(n_target_plus))
    p_target_minus = np.random.multivariate_normal(mean=np.array([2,1]), cov=np.array([[2,1], [1,2]]), size=(n_target_minus))

    p_target = np.vstack((p_target_plus, p_target_minus))

    return p_source_plus, p_source_minus, p_target

In [5]:
def generate_sample_fig2(scale):
    n_source_plus = int(150*scale)
    n_source_minus = int(200*scale)
    n_source = n_source_plus + n_source_minus
    pi_source = n_source_plus/n_source

    n_target_plus = int(100*scale)
    n_target_minus = int(150*scale)
    n_target = n_target_plus + n_target_minus
    pi_target = n_target_plus/n_target

    p_source_plus = np.random.multivariate_normal(mean=np.array([0,0]), cov=np.eye(2), size=(n_source_plus))
    p_source_minus = np.random.multivariate_normal(mean=np.array([2,1]), cov=np.array([[2,1], [1,2]]), size=(n_source_minus))

    p_target_plus = np.random.multivariate_normal(mean=np.array([0,0]), cov=np.eye(2), size=(n_target_plus))
    p_target_minus = np.random.multivariate_normal(mean=np.array([2,1]), cov=np.array([[2,1], [1,2]]), size=(n_target_minus))

    p_target = np.vstack((p_target_plus, p_target_minus))

    return p_source_plus, p_source_minus, p_target

In [6]:
p_source_plus, p_source_minus, p_target = generate_sample_fig2(1)
p_source = np.vstack((p_source_plus, p_source_minus))
y_source = np.concat((np.ones(p_source_plus.shape[0]), np.zeros(p_source_minus.shape[0])))

In [7]:
df_source = pd.DataFrame(p_source)
df_source.columns = ['x1', 'x2']
df_target = pd.DataFrame(p_target)
df_target.columns = ['x1', 'x2']
df_y_source = pd.DataFrame({'response': y_source})

In [8]:
p_source.shape, y_source.shape

((350, 2), (350,))

In [9]:
extra = robjects.ListVector({'n.tree': robjects.IntVector([100])})

In [10]:
# with localconverter(robjects.default_converter + pandas2ri.converter + numpy2ri.converter):
#     a = g_forman_random(y_source, df_source, df_target, extra)

In [11]:
g_kernel_leave = robjects.globalenv['g.kernel.leave']

In [12]:
extra = robjects.ListVector({'n.tree': robjects.IntVector([100])})

In [13]:
# with localconverter(robjects.default_converter + pandas2ri.converter + numpy2ri.converter):
#     a = g_kernel_leave(y_source, df_source, df_target)

In [14]:
gaussian_kernel = robjects.globalenv['gaussian.kernel']

In [15]:
gaussian_kernel = robjects.r['gaussian.kernel']

In [21]:
g_list = robjects.ListVector({
    'first_method': robjects.ListVector({
        'func':  robjects.r['g.kernel.leave'], # robjects.StrVector(['g.kernel.leave'])
        'extra': robjects.ListVector({
            'lambda': robjects.FloatVector([0.001]),
            'kernel': robjects.r['gaussian.kernel']
    })
    }),
    'second_method': robjects.ListVector({
        'func':  robjects.r['g.forman.random'], # robjects.StrVector(['g.kernel.leave'])
        'extra':robjects.ListVector({'n.tree': robjects.IntVector([100])})
    })
})


In [22]:
quantification_prior_shift = robjects.globalenv['quantification.prior.shift']

In [23]:
with localconverter(robjects.default_converter + pandas2ri.converter + numpy2ri.converter):
    a = quantification_prior_shift(df_source, y_source, df_target, g_list)

[1] "first"
[1] "first"
[1] "loop"
[1] 1
[1] "loop"
[1] 1
[1] "loop"
[1] 2


In [24]:
a[0]

rec.array([('first_method', 0.34740263, 0.2553258, 0.12186855, 0.50602583, 0.03978192, 0.05731168, 0.00303668),
           ('second_method', 0.34693878, 0.42     , 0.25      , 0.74      , 0.18844221, 0.19369128, 0.00639701)],
          dtype=[('method', '<U13'), ('estimative', '<f8'), ('mu_g', '<f8'), ('mu_g0', '<f8'), ('mu_g1', '<f8'), ('var_g0', '<f8'), ('var_g1', '<f8'), ('var', '<f8')])

In [25]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import itertools

from kernels_class import *
from rbf_kernel import *
from generation import *

In [26]:
est_tmp = estimator_RHKS(p_target, p_source_plus, p_source_minus, kernel_params={'gamma': 1/2})
est_tmp.estimate_pi_nrm()
est_tmp.estimate_pi_ipr()

est_tmp.pi_nrm, est_tmp.pi_ipr

(np.float64(0.3684441819260186), np.float64(0.3736160455033683))

In [ ]:
est_tmp.compute_K2()
est_tmp.compute_tau_plug_in()
est_tmp.compute_tau_explicit()

est_tmp.estimate_variance_explicit()
est_tmp.estimate_variance_plug_in()

est_tmp.var_plug_in*est_tmp.r_n, est_tmp.var_explicit*est_tmp.r_n

(np.float64(0.003201927576972918), np.float64(0.19609569046826916))